<a href="https://colab.research.google.com/github/19K41A0577/NNDL/blob/main/assignment3_NNDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title MOUNT GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#@title IMPORT LIBRARIES
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [15]:
#@title LOAD THE DATA
data = pd.read_excel('/content/drive/MyDrive/Assignment3.xlsx')
print(data.shape)
data.head()

(2184, 3)


,Date,Time,Load (kW)
0,01.09.2018,00:00:00,5551.82208
1,NaN,01:00:00,4983.17184
2,NaN,02:00:00,4888.39680
3,NaN,03:00:00,5072.95872
4,NaN,04:00:00,5196.25980


In [16]:
x = []
y = []
for i in range(1, data.shape[0]):
    x.append(data['Load (kW)'].iloc[i-1]) 
    y.append(data['Load (kW)'].iloc[i])

In [17]:
data = pd.DataFrame({'Previous Hour Load' : x, 'Present Hour Load' : y})
data.head()

,Previous Hour Load,Present Hour Load
0,5551.82208,4983.17184
1,4983.17184,4888.39680
2,4888.39680,5072.95872
3,5072.95872,5196.25980
4,5196.25980,5641.29720


In [19]:
#@title DATA NORMALISATION
maxX = np.max(data['Previous Hour Load'])
minX = np.min(data['Previous Hour Load'])
maxY = np.max(data['Present Hour Load'])
minY = np.min(data['Present Hour Load'])

data['Previous Hour Load'] = (data['Previous Hour Load'] - minX) / (maxX - minX)
data['Present Hour Load'] = (data['Present Hour Load'] - minY) / (maxY - minY)

In [20]:
#@title SPLIT THE DATA
x = data['Previous Hour Load']
y = data['Present Hour Load']
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.1, random_state=10)

In [23]:
#@title ACCELERATED GRADIENT (NAG) +SGD OPTIMIZER
m = np.random.uniform(-5, 5)
c = np.random.uniform(-5, 5)
eta = 0.01
epochs = 500
vm = 0
vc = 0
v = 0.1
for _ in range(epochs):   
    for i in range(trainX.shape[0]):
        tempM = m + v*vm
        tempC = c + v*vc       
        gradM = -1 * (trainY.iloc[i] - tempM * trainX.iloc[i] - tempC) * trainX.iloc[i]
        gradC = -1 * (trainY.iloc[i] - tempM * trainX.iloc[i] - tempC)
        vm = (v * vm) - (eta * gradM)
        vc = (v * vc) - (eta * gradC)
        m += vm
        c += vc
print(f'm = {m}\nc = {c}')

m = 0.7794148724880392
c = 0.09669307635526696


In [24]:
train_pred = [m * trainX.iloc[i] + c for i in range(trainX.shape[0])]
test_pred = [m * testX.iloc[i] + c for i in range(testX.shape[0])]

train_pred = [i * (maxY - minY) + minY for i in train_pred]
test_pred = [i * (maxY - minY) + minY for i in test_pred]
trainY = [i * (maxY - minY) + minY for i in trainY]
testY = [i * (maxY - minY) + minY for i in testY]

training_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
training_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
training_RMSE = training_MSE ** 0.5
print('Training Error :')
print(f'MAE : {training_MAE}')
print(f'MSE : {training_MSE}')
print(f'RMSE : {training_RMSE}')

testing_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
testing_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
testing_RMSE = testing_MSE ** 0.5
print('Testing Error :')
print(f'MAE : {testing_MAE}')
print(f'MSE : {testing_MSE}')
print(f'RMSE : {testing_RMSE}')

Training Error :
MAE : 0.08513992443105549
MSE : 0.015002198153347355
RMSE : 0.12248346073387768
Testing Error :
MAE : 0.08515284128807092
MSE : 0.0138792015503937
RMSE : 0.11781002313213294


In [25]:
pd.DataFrame({'Actual' : trainY, 'Predicted' : train_pred})

,Actual,Predicted
0,0.236798,0.270361
1,0.312801,0.407138
2,0.743659,0.488991
3,0.759322,0.552766
4,0.068957,0.184995
...,...,...
1959,0.247924,0.218994
1960,0.355111,0.569799
1961,0.445636,0.572223
1962,0.171607,0.273163


In [26]:
x = float(input('Enter the load at previous hour : '))
x = (x - minX) / (maxX - minX)
prediction = m * x + c
prediction = (prediction * (maxY - minY)) + minY
print('Predicted load at present hour :', prediction)

Enter the load at previous hour : 5789.8
Predicted load at present hour : 4512.752921807605
